In [1]:
import os

In [2]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Plant-Disease-Classification-using-CNN\\research'

In [3]:
os.chdir('..')

In [4]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Plant-Disease-Classification-using-CNN'

## Entity

In [5]:
import os
from dataclasses import  dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    INCLUDE_TOP: bool
    INPUT_SHAPE: list
    WEIGHTS: str
    CLASSES: int
    OPTIMIZER: str
    LOSS: str
    METRICS: list
    

## Config Manager

In [7]:
from plant_disease_clf.utils.common import  create_directories, read_yaml
from plant_disease_clf.constants import  *

In [23]:
class ConfigManager:
    def __init__(self,
    config_file_path = CONFIG_FILE_PATH,
    params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:

        self.config = self.config.base_model
        self.params = self.params.resnet50

        create_directories([self.config.root_dir])

        return PrepareBaseModelConfig(
            root_dir = self.config.root_dir,
            base_model_path = self.config.base_model_path,
            updated_model_path = self.config.updated_model_path,
            INCLUDE_TOP = self.params.INCLUDE_TOP,
            INPUT_SHAPE = self.params.INPUT_SHAPE,
            WEIGHTS = self.params.WEIGHTS,
            CLASSES = self.params.CLASSES,
            LOSS = self.params.LOSS,
            METRICS = self.params.METRICS,
            OPTIMIZER = self.params.OPTIMIZER,
        )

    


## Components

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow


In [10]:
import os,sys
from pathlib import  Path

from plant_disease_clf.logger import  logging
from plant_disease_clf.exception import  CustomException

import tensorflow as tf


In [25]:
import keras
from keras.layers import Conv2D, Flatten, Dense, Input, Lambda, Dropout
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50

from keras.saving import  load_model

In [15]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def base_model(self): ## ResNet50 model with imagenet weights
        try:
            model = ResNet50(
                include_top = self.config.INCLUDE_TOP,
                input_shape = self.config.INPUT_SHAPE,
                weights = self.config.WEIGHTS
            )

            for layer in model.layers:
                layer.trainable = False
            
            model.summary()

            model.save(self.config.base_model_path)

            return model
        
        except Exception as e:
            raise CustomException(e,sys)
        


    def updated_base_model(self): ## Top layers will be added here
        try:

            self.base_model = self.base_model()

            self.full_model = Sequential()
            self.full_model.add(self.base_model)
            self.full_model.add(Flatten())

            self.full_model.add(Dense(256, activation='relu'))
            self.full_model.add(Dropout(0.4))
            self.full_model.add(Dense(self.config.CLASSES, activation='softmax'))

            self.full_model.summary()

            self.full_model.compile(
                optimizer = self.config.OPTIMIZER,
                loss = self.config.LOSS,
                metrics = self.config.METRICS
            )

            self.full_model.save(self.config.updated_model_path)

            return self.full_model

        except Exception as e:
            raise CustomException(e,sys)


    # def save_model()

## Pipeline

In [26]:
try:
    config = ConfigManager()
    config = config.get_prepare_base_model_config()
    PrepareBaseModel(config).updated_base_model()
except Exception as e:
    raise CustomException(e,sys)

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       